In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from PIL import Image
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage.color import label2rgb
from scipy import ndimage, spatial
from scipy.spatial import distance as dist
from scipy.stats import wasserstein_distance

import tensorflow as tf

In [ ]:
path_train = 'lbp_data/patterndata/train'
path_test = 'lbp_data/patterndata/test'

In [ ]:
radius=5
num_pts=25
match=0

In [ ]:
lbp_train_features = []
train_data = []
for filename in os.listdir(path_train):
    image_train = os.path.join(path_train, filename)
    print(image_train)
    train_img = Image.open(image_train)
    train_arr = np.asarray(train_img)
    train_gray = cv2.cvtColor(train_arr, cv2.COLOR_BGR2GRAY)
    train_lbp = local_binary_pattern(train_gray, num_pts, radius,
                                    'uniform')
    lbp_train_features.append(train_lbp)
    train_data.append(filename)

In [ ]:
for filename_test in os.listdir(path_test):
    image_test = os.path.join(path_test, filename_test)
    test_img = Image.open(image_test)
    test_arr = np.asarray(test_img)
    test_gray = cv2.cvtColor(train_arr, cv2.COLOR_BGR2GRAY)
    lbp_test_features = local_binary_pattern(test_gray, num_pts, radius,
                                            'uniform')
    bins_num = int(lbp_test_features.max() + 1)
    test_hist, _ = np.histogram(lbp_test_features, bins=bins_num, 
                                range=(0, bins_num), density=None)
    my_met=0
    match= 0
    q = []
    min_score = 10000
    
    fig = plt.figure()
    fig.subplots_adjust(wspace=.5)
    
    ax1 = fig.add_subplot(231)
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    plt.xticks([])
    plt.yticks([])
    ax1.set_title(filename_test, fontsize=15, fontweight='bold')
    plt.subplot(231), plt.imshow(test_img)
    fig.subplots_adjust(hspace=.5)
    plt.subplot(232), plt.plot(test_hist)
    
    ax3 = fig.add_subplot(233)
    ax3.set_xticklabels([])
    ax3.set_yticklabels([])
    plt.xticks([])
    plt.yticks([])
    plt.subplot(233), plt.imshow(lbp_test_features, cmap ='gray')

    ax2 = fig.add_subplot(234)
    ax2.set_xticklabels([])
    ax2.set_yticklabels([])
    plt.xticks([])
    plt.yticks([])
    
    ax4 = fig.add_subplot(236)
    ax4.set_xticklabels([])
    ax4.set_yticklabels([])
    plt.xticks([])
    plt.yticks([])
    
    for features in lbp_train_features: 
        my_met=my_met+1
        histogram, _ = np.histogram(features, bins=bins_num, 
                                    range=(0, bins_num), density=None)
        p = np.asarray(test_hist)
        q = np.asarray(histogram)
        filter_idx = np.where(np.logical_and(p != 0, q != 0))

        minima = np.minimum(test_hist,histogram)
        intersection = np.true_divide(np.sum(minima),np.sum(histogram))
        difidx = p-q
        score = .5*(np.sum((p-q)**2/(p+q+1e-10)))
        score_was = wasserstein_distance(p,q)
        score_chisq = .5*(np.sum((p-q)**2/(p+q+1e-10)))
        score_simple = np.sum((p-q)**2)
        score_eucl = dist.euclidean(p,q)
        score_man = dist.cityblock(p,q)

        if score < min_score:
            min_score = score
            match = my_met-1
            
    dir_train = os.path.join(path_train, train_data[match])
    matched_img = Image.open(dir_train)
    matched_train_arr = np.asarray(matched_img)
    matched_train_gray = cv2.cvtColor(matched_train_arr, cv2.COLOR_BGR2GRAY)
    matched_train_lbp = local_binary_pattern(matched_train_gray, num_pts, radius, 
                                             'uniform')
    
    print("\033[1m" + "Match pattern: ", train_data[match], filename_test + "\033[0m")
    plt.subplot(234),plt.imshow(matched_img)
    ax2.set_title(train_data[match], fontsize=15, fontweight = 'bold')
    plt.subplot(236),plt.imshow(matched_train_lbp, cmap ='gray')
    
    plt.subplot(235),plt.plot(histogram) 
    plt.show()

In [ ]:
path_face = 'lbp_data/facedata/facecolr'
path_found = 'lbp_data/facedata/foundcolr'

In [ ]:
found_img = []
found_colr = []
for filename_found in os.listdir(path_found):
    image_found = os.path.join(path_found, filename_found)
    print(image_found)
    foundation_img = Image.open(image_found)
    found_arr = np.asarray(foundation_img)
    found_colr.append(found_arr)
    found_img.append(filename_found)

In [ ]:
for filename_face in os.listdir(path_face):
    image_face = os.path.join(path_face, filename_face)
    face_img = Image.open(image_face)
    face_arr = np.asarray(face_img)
    (mfc, sfc) = cv2.meanStdDev(face_arr)
    statfc = np.concatenate([mfc, sfc]).flatten()
    face_bchanne1 = statfc[0]
    face_gchannel = statfc[1]
    face_rchannel = statfc[2]
    
    min_sim=10000
    sim_idx=0
    match_idx=0
    fig_shade = plt.figure()
    fig_shade.subplots_adjust(wspace=.5)
    
    axshd1 = fig_shade.add_subplot(121)
    axshd1.set_xticklabels([])
    axshd1.set_yticklabels([])
    plt.xticks([])
    plt.yticks([])
    axshd1.set_title(filename_face, fontsize=15, fontweight = 'bold')
    plt.subplot(121), plt.imshow(face_img) 
    
    axshd2 = fig_shade.add_subplot(122)
    axshd2.set_xticklabels([])
    axshd2.set_yticklabels([])
    plt.xticks([])
    plt.yticks([])
    
    for shades in found_colr: 
        sim_idx +=1
        (mfnd, sfnd) = cv2.meanStdDev(shades)
        statfnd = np.concatenate([mfnd, sfnd]).flatten()
        found_bchannel=statfnd[0]
        found_gchannel=statfnd[1]
        found_rchannel=statfnd[2]
        dif_face_fnd = abs(0.299*(face_rchannel - found_rchannel) 
                           + 0.587*(face_gchannel - found_gchannel)
                           + 0.114*(face_bchanne1 - found_bchannel))
        
        if dif_face_fnd < min_sim:
            min_sim = dif_face_fnd
            match_idx = sim_idx-1
            
    dir_found = os.path.join(path_found, found_img[match_idx])
    matched_fnd = Image.open(dir_found)
    
    print("\033[1m" + "Match pattern: ", found_img[match_idx], filename_face + "\033[0m")

    axshd2.set_title(found_img[match_idx], fontsize=15, fontweight = 'bold')
    plt.subplot(122),plt.imshow(matched_fnd)
    
    plt.show()